# Overview of Prompt Engineering Techniques & Best Practices

## Part 2: Prompt Engineering Techniques

In this section, we cover the best prompt engineering techniques and how to apply them.

---

We first load the necessary libraries:

In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', '')))

In [2]:
import IPython
from src.utils import setup_credentials, get_completion

In [3]:
setup_credentials()

In [4]:
# ! pip install openai==0.28 langchain google-search-results chromadb tiktoken --quiet

### Few-shot In-Context Learning

Below we provide an example of few-shot prompting with demonstrations:

In [5]:
system_message = """Your task is to classify an user text input as either offensive or non-offensive.

Use the following examples to help with steering your responses:

Input: I love you
Output: non-offensive

Input: I dislike all those people working at the company
Output: offensive

Input: I think this feature is not ideal
Output: non-offensive

Input: Those people are so stupid
Output: offensive
"""

user_message = "I respectfully disagree with your opinion."

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages)
print(response)

non-offensive 



### Chain-of-Though (CoT) Prompting

Below is an example of CoT applied. Specifically, we aim to build a movie recommendation system using CoT.

Let's first define a set of movies:

In [6]:
movies = """
The Enigma Code
Category: Historical Drama
Rating: 8.3/10
Description: Set during World War II, this gripping historical drama follows the life of Alan Turing, a brilliant mathematician tasked with cracking the Enigma code used by the Nazis. His efforts contribute significantly to the Allies' victory.
Actors: Benedict Cumberbatch, Keira Knightley, Matthew Goode
Language: English
Release Date: March 15, 2014
Award Winner: Academy Award for Best Adapted Screenplay

Shadows of the Samurai
Category: Action/Adventure
Rating: 7.9/10
Description: In feudal Japan, a skilled samurai seeks vengeance against the corrupt warlord who murdered his master. With his swordsmanship and determination, he embarks on a dangerous journey to restore justice.
Actors: Ken Watanabe, Tadanobu Asano, Rinko Kikuchi
Language: Japanese
Release Date: November 7, 2017
Award Winner: None

Mind Games
Category: Psychological Thriller
Rating: 8.1/10
Description: A renowned psychologist becomes entangled in a twisted game of cat and mouse with a patient who harbors dark secrets. As their sessions progress, the lines between reality and deception blur, leading to a mind-bending climax.
Actors: Leonardo DiCaprio, Natalie Portman, Michael Fassbender
Language: English
Release Date: August 22, 2019
Award Winner: None

La Casa del Tango
Category: Musical/Drama
Rating: 8.7/10
Description: In the vibrant world of Buenos Aires, a passionate tango dancer finds love and inspiration amidst the backdrop of political unrest. This musical drama explores the power of dance and the pursuit of dreams.
Actors: Antonio Banderas, Penélope Cruz, Javier Bardem
Language: Spanish
Release Date: June 5, 2020
Award Winner: Golden Globe for Best Foreign Language Film

Timeless Love
Category: Romance/Fantasy
Rating: 7.5/10
Description: A magical encounter transports a modern-day writer back in time to Victorian England, where she falls in love with a charming aristocrat. As they navigate the complexities of time, their love is put to the ultimate test.
Actors: Rachel McAdams, Tom Hiddleston, Emma Thompson
Language: English
Release Date: February 14, 2022
Award Winner: None

The Pursuit of Justice
Category: Legal Drama
Rating: 8.4/10
Description: Inspired by true events, this gripping legal drama follows a determined lawyer's fight against a powerful pharmaceutical company responsible for a life-threatening drug. The courtroom battle becomes a quest for justice and redemption.
Actors: Denzel Washington, Viola Davis, Michael B. Jordan
Language: English
Release Date: October 10, 2022
Award Winner: None

The Forgotten Island
Category: Adventure/Mystery
Rating: 7.6/10
Description: A group of explorers stumbles upon a mysterious island believed to be uninhabited. As they uncover the island's secrets, they encounter deadly challenges and unravel an ancient civilization's enigma.
Actors: Chris Pratt, Bryce Dallas Howard, Tom Holland
Language: English
Release Date: July 2, 2023
Award Winner: None

The Silent Witness
Category: Crime/Thriller
Rating: 8.2/10
Description: A talented forensic pathologist becomes entangled in a high-stakes murder investigation when she discovers crucial evidence that points to a powerful criminal network. With her life on the line, she must outsmart the perpetrators.
Actors: Emily Blunt, Jake Gyllenhaal, Mark Ruffalo
Language: English
Release Date: November 18, 2023
Award Winner: None

A Tale of Two Worlds
Category: Fantasy/Adventure
Rating: 7.8/10
Description: When a young orphan discovers a magical portal to a parallel universe, she embarks on a thrilling adventure to save both realms from an impending disaster. Along the way, she learns about the power of friendship and self-belief.
Actors: Millie Bobby Brown, Tom Holland, Helena Bonham Carter
Language: English
Release Date: April 5, 2024
Award Winner: None

A Symphony of Souls
Category: Music/Drama
Rating: 9.0/10
Description: Set against the backdrop of a renowned symphony orchestra, this emotionally charged drama explores the lives and intertwining stories of its members. Through the power of music, they find solace, love, and redemption.
Actors: Meryl Streep, Tom Hanks, Cate Blanchett
Language: English
Release Date: December 25, 2024
Award Winner: None
"""

We then take those movies and list the different steps the model should perform. Notice that the after detailing the steps, we ask the model to providing the reasoning steps and the final response. This is a format you can use to elicit reasoning in LLMs.

In [7]:
# the system message contains the logic (step by step) for the system to follow
system_message = """
You task is to make movie recommendations based on a user request (delimited by ```).

Step 1: Check if the user is asking about movies. If the user is not asking about movies, just respond "Please ask something about movies!".

Step 2: If the user is asking for a movie recommendation, check if they have any specific requests or interests.

Step 3: Check if there are any movie/s we can recommend from the following: {movies}

Step 4: Prepare a response to the user with the movie recommendation/s. The recommendation have to be about movies that are available in the list above. The response needs to have a friendly and helpful tone.

Return a response with the following reasoning steps and final output to the user:
Step 1: <Step 1 reasoning>
Step 2: <Step 2 reasoning>
Step 3: <Step 3 reasoning>
Step 4: <final response>
"""

messages = [
    {
        "role": "system",
        "content": system_message.format(movies=movies)
    },
    {
        "role": "user",
        "content": "```Do you have any drama movies?```"
    }
]

movie_recommendation_response = get_completion(messages)

print(movie_recommendation_response)

Step 1: The user is asking about movies.
Step 2: The user is asking for a drama movie recommendation.
Step 3: We have several drama movies available: "The Enigma Code", "La Casa del Tango", "The Pursuit of Justice", and "A Symphony of Souls".
Step 4:  Sure! We have a few drama movies you might enjoy. "The Enigma Code" is a historical drama set during World War II, "La Casa del Tango" is a musical drama set in Buenos Aires, "The Pursuit of Justice" is a legal drama inspired by true events, and "A Symphony of Souls" is a music drama set in a symphony orchestra. Which one sounds most interesting to you? 



### Prompt Chaining

The example below demonstrates how to chain separate prompts to achieve a specific task. The previous prompt list all the reasoning steps. In the following prompt, we ask the model to extract only the final response to the user:

In [8]:
# Prompt 1: step by step reasoning (provided above)
# Prompt 2: extract only the final response we will send to the user

system_message_2 = """
You will be given a list of steps that a model has responded with. Your task is to extract only the full response in Step 4 from the following text: {movie_recommendation_response}

Step 4:
"""

messages = [
    {
        "role": "user",
        "content": system_message_2.format(movie_recommendation_response=movie_recommendation_response)
    }
]

final_response = get_completion(messages)

print(final_response)

Sure! We have a few drama movies you might enjoy. "The Enigma Code" is a historical drama set during World War II, "La Casa del Tango" is a musical drama set in Buenos Aires, "The Pursuit of Justice" is a legal drama inspired by true events, and "A Symphony of Souls" is a music drama set in a symphony orchestra. Which one sounds most interesting to you? 



### ReAct

The code below shows an example of combining LLMs with external tools to achieve a task. In particular, the example uses the ReAct framework to prompt and guide the model to the result by leveraging the LLM and a search engine. We will be using SerpAPI in the below example. As mentioned in the course introduction, you will need to register for a free account with SerpAPI to complete this part of the exercise. You can [register here](https://serpapi.com/).

In [9]:
from langchain.agents import load_tools
from langchain.agents import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent

In [10]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", temperature=0.0, max_output_tokens=1024
)

In [ ]:
# tools and external agent
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm , agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

/tmp/ipykernel_143076/1720539967.py:3: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(tools, llm , agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [12]:
# run the agent
agent.invoke("Which team won the 2023 NBA finals?")



> Entering new AgentExecutor chain...
Thought: I need to find out who won the 2023 NBA finals.
Action: Search
Action Input: 2023 NBA finals winner
Observation: Game Recap: Nuggets 94, Heat 89 After closing out this best-of-seven series, 4-1, the Nuggets have clinched the franchises first NBA title.
Thought:Thought: I now know the final answer
Final Answer: The Denver Nuggets won the 2023 NBA finals. 


> Finished chain.


{'input': 'Which team won the 2023 NBA finals?',
 'output': 'The Denver Nuggets won the 2023 NBA finals.'}

### Data Augmentation / RAG

The example below demonstrates how to apply data augmented generation.

We first load the necessary LangChain modules:

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

from langchain.chains.qa_with_sources import load_qa_with_sources_chain

Next, we split the loaded text file into chunks and embed the chunks using OpenAI Embeddings.

In [14]:
# load the data from GitHub
import urllib
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/kar-gpt.txt")
text_data = f.read().decode("utf-8")

In [15]:
# split text into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator=" ")
texts = text_splitter.split_text(text_data)

In [ ]:
# init LLM from Google Generative AI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", temperature=0.0, max_output_tokens=1024
)

In [17]:
# embeddings obtained from Google Generative AI
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

The next step is to store the embeddings into Chroma, a dedicated vector storage. And finally, we perform a search by querying Chroma.

In [ ]:
# vector storage for embeddings
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

query = "What is the course about?"
docs = docsearch.similarity_search(query)

chain = load_qa_with_sources_chain(llm, chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

/tmp/ipykernel_143076/3492953869.py:7: LangChainDeprecationWarning: This function is deprecated. Refer to this guide on retrieval and question answering with sources: https://python.langchain.com/v0.2/docs/how_to/qa_sources/
See also the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

  chain = load_qa_with_sources_chain(llm, chain_type="stuff")
/tmp/ipykernel_143076/3492953869.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"input_documents": docs, "question": query}, return_only_outputs=Tru

{'output_text': "FINAL ANSWER: The course is about training a transformer-based language model, specifically a character-level language model, using Shakespeare's works as a dataset. \nSOURCES: 3, 4, 94, 108 \n"}